In [25]:
import pandas as pd
import numpy as np
# import yfinance
import tqdm

# from mplfinance import candlestick_ohlc
from mplfinance.original_flavor import candlestick_ohlc

import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt
import os
import plotly.graph_objects as go

from datetime import datetime

In [26]:
def isSupport(df,i):
  # fractal (i-2)>(i-1)>i<(i+1)<(i+2)
  support = df['Low'][i] < df['Low'][i-1]  and df['Low'][i] < df['Low'][i+1] \
  and df['Low'][i+1] < df['Low'][i+2] and df['Low'][i-1] < df['Low'][i-2]

  return support

def isResistance(df,i):
  resistance = df['High'][i] > df['High'][i-1]  and df['High'][i] > df['High'][i+1] \
  and df['High'][i+1] > df['High'][i+2] and df['High'][i-1] > df['High'][i-2] 

  return resistance

In [27]:
def isFarFromLevel(l):
  return np.sum([abs(l-x) < s  for x in levels]) == 0

In [28]:

name = 'BTC-USD'
dataframes={}
timeframe='4h'
outdir2 = f"{os.getcwd()}/data/{timeframe}"


for filename in tqdm(os.listdir(outdir2)):
    symbol = filename.split("_")[0]
    df = pd.read_csv(f"{outdir2}/{filename}",index_col=['Time'], parse_dates=['Time'])

    df['Date'] = pd.to_datetime(df.index)
    # change datetime into matplotlib time
    # does not work with plotly
    # df['Date'] = df['Date'].apply(mpl_dates.date2num)

    df = df.loc[:,['Date' ,'Open', 'High', 'Low', 'Close']]

    levels = []
    # start from 2 because i need i-2
    # end at shape[0]-2 because  i need i+2
    # i is the number of current row
    for i in range(2,df.shape[0]-2):
        if isSupport(df,i):
            levels.append((i,df['Low'][i]))
        elif isResistance(df,i):
            levels.append((i,df['High'][i]))


    s =  np.mean(df['High'] - df['Low'])

    levels = []
    for i in range(2,df.shape[0]-2):
        if isSupport(df,i):
            l = df['Low'][i]

            if isFarFromLevel(l):
                levels.append((i,l))

        elif isResistance(df,i):
            l = df['High'][i]

            if isFarFromLevel(l):
                levels.append((i,l))

    for level in levels:
        if (df['Close'][-2] < level[1]) and (df['Close'][-1] > level[1]):
            print(f"{level[1]} breakout {symbol}")
    
    dataframes[symbol] = df



55.68 breakout AXSUSDT
0.3677 breakout BEAMUSDT
0.1692 breakout BLZUSDT
32.81 breakout BTGUSDT
1.938 breakout ENJUSDT
6.69 breakout FLOWUSDT
0.258 breakout GALAUSDT
0.03779 breakout GTOUSDT
0.259 breakout HBARUSDT
28.11 breakout HNTUSDT
3.675 breakout KAVAUSDT
0.4866 breakout KEEPUSDT
2.075 breakout KNCUSDT
0.22801 breakout LTOUSDT
0.007376 breakout MFTUSDT
0.04054 breakout MITHUSDT
1.082 breakout PLAUSDT
0.614 breakout POWRUSDT
5.52 breakout RADUSDT
15.4 breakout REPUSDT
0.9 breakout SFPUSDT
2.333e-05 breakout SHIBUSDT
0.01665 breakout STMXUSDT
0.7235 breakout TRIBEUSDT
0.2562 breakout UTKUSDT


In [29]:
def plot_all(df):

    candlestick = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])

    # fig = go.Figure(data=[go.Candlestick(x=df['Date'],
    #             open=df['Open'],
    #             high=df['High'],
    #             low=df['Low'],
    #             close=df['Close'])])
    fig = go.Figure(data=candlestick)

    for level in levels:
        # fig.add_hline(level[1])
        fig.add_shape(type='line',
                x0=df['Date'][level[0]],
                y0=level[1],
                x1=max(df['Date']),
                y1=level[1],
                line=dict(color='black',),
                xref='x',
                yref='y')
    
    fig.layout.xaxis.rangeslider.visible = False


    fig.show()

In [31]:
df = dataframes['SFPUSDT']
plot_all(df)